In [14]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Inicializa MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)


def calculate_angle(a, b, c):
    """Calcula el ángulo entre tres puntos clave."""
    a = np.array([a.x, a.y])  # Primer punto
    b = np.array([b.x, b.y])  # Punto central (vértice)
    c = np.array([c.x, c.y])  # Tercer punto

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

class PressBancaCounter:
    def __init__(self):
        self.reps = 0
        self.state = "up"  # Estado inicial: en extensión completa

    def update(self, left_angle, right_angle):
        # Umbrales para los ángulos
        min_flex_angle = 80  # Punto máximo de flexión
        max_extend_angle = 100  # Punto máximo de extensión

        # Determinar el estado actual basado en los ángulos
        if left_angle >= max_extend_angle and right_angle >= max_extend_angle:
            if self.state == "down":
                self.reps += 1  # Contar repetición cuando se vuelva a extensión
                
            self.state = "up"
        elif left_angle <= min_flex_angle and right_angle <= min_flex_angle:
            self.state = "down"

        return self.reps
class Rm:
    def __init__(self, weight):
        self.rm = 0.0
        self.weight = weight   
           
    def Epley(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = (self.weight*newReps*0.033)+self.weight
        return self.rm
    
    def Brzycki(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = self.weight/(1.0278 - (0.0278*newReps))
        return self.rm
    
    def EpleyReducida(self, reps):
        newReps = self.RepsToInt(reps)
        work = 0.03
        suma = 0
        for i in range(5):
            suma += (self.weight*newReps*work)+self.weight
            work -= 0.001
        suma = suma/5
        self.rm = round(suma, 1)
        return self.rm
    
    def RmVelocidad(self, reps, velocidades):
        newReps = self.RepsToInt(reps)
        suma = 0
        
        for velocidad in velocidades:
            suma += self.weight / (1 - (velocidad * 0.03))
        suma = suma/len(velocidades)
        self.rm = round(suma,1)
        return self.rm    
        """#hacer el cambio de la formula
        VR = max(velocidad / 0.278, 0.1)
        suma += (self.weight * (1 + 0.0333 * newReps)) * (1 / VR)
    suma = suma/len(velocidades)
    self.rm = round(suma,1)
    return self.rm"""
    
    def RepsToInt(self, reps):
        return float(reps.split(':')[-1].strip())
    
class VelocityCalculator:
    def __init__(self):
        self.last_time = None
        self.velocities = []
        self.fps_original = 0
        self.actual_fps = 0
        self.frame_high = 0
        # Nuevos atributos para tiempo transcurrido
        self.start_time = None  # Tiempo en posición baja
        self.end_time = None  # Tiempo en posición alta
        self.times = []  # Lista para almacenar los tiempos transcurridos entre eventos
        self.distancia = 0.47

    def update(self, tiempo_estimado):
        """
        Calcula y almacena la velocidad basada en el tiempo entre eventos.
        """
        if tiempo_estimado > 0:  # Prevenir divisiones por cero
            velocidad = 1 / tiempo_estimado  # Velocidad proporcional al tiempo
            velocidad_ajustada = velocidad * (self.fps_original / self.actual_fps)
            self.velocities.append(velocidad)
            return velocidad_ajustada
            #esto es para el caso de utilizar una medida de referencia (longitud del brazo)
            """velocidad = self.distancia / tiempo_estimado 
            self.velocities.append(velocidad)
            return velocidad"""

        #return None

    def start_timing(self, wrist_position_y):
        """Inicia el temporizador y registra la posición de la muñeca en el punto bajo."""
        self.start_time = time.time()
        self.start_position_y = wrist_position_y * self.frame_high
        
    def stop_timing(self, wrist_position_y):
        """Detiene el temporizador y registra la posición de la muñeca en el punto alto."""
        self.end_time = time.time()
        if self.start_time is not None and self.end_time is not None:
            elapsed_time = self.end_time - self.start_time

            
            corrected_elapsed_time = (elapsed_time * self.actual_fps) / self.fps_original
            # Almacena el tiempo corregido para análisis posterior
            self.times.append(corrected_elapsed_time)

            self.start_time = None  # Reinicia para la próxima repetición
            return corrected_elapsed_time

        return None, None, None

    def get_average_time(self):
        """Calcula el tiempo promedio entre eventos registrados."""
        if self.times:
            return np.mean(self.times)
        return 0.0

    def get_average_velocity(self):
        """Calcula la velocidad media de todas las repeticiones registradas."""
        if self.velocities:
            return np.mean(self.velocities)
        return 0.0

    def set_actual_fps(self, actual_fps):
        """Establece el FPS actual, útil para ajustar las velocidades cuando se ralentiza el video."""
        self.actual_fps = actual_fps

class PressBancaState: 
    def __init__(self):
        self.puntuacion_extension = 0.0
        self.puntuacion_bajo = 0.0
        self.puntuacion_rom = 0.0
        self.repeticion = 0
        self.state = "up"  # Estado inicial (extensión máxima)
        self.tiempo_estimado = 0
        self.ROM = []

    def calculate_extension_score(self, left_angle, right_angle):
        """Calcula la puntuación de la extensión en el punto máximo."""
        def score_angle(angle):
            if 150 <= angle <= 160:
                return 9 + (angle - 150) / 10  # Escala 9-10
            elif 130 <= angle < 150:
                return 7 + (angle - 130) / 20 * 2  # Escala 7-8
            elif 110 <= angle < 130:
                return 4 + (angle - 110) / 20 * 2  # Escala 4-6
            elif angle < 110:
                return max(0, (angle - 100) / 10 * 3)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def calculate_contraction_score(self, left_angle, right_angle):
        """Calcula la puntuación de la contracción en el punto máximo."""
        def score_angle(angle):
            if 30 <= angle <= 40:
                return 9 + (40 - angle) / 10  # Escala 9-10
            elif 45 <= angle <= 55:
                return 7 + (55 - angle) / 10 * 2  # Escala 7-8
            elif 60 <= angle <= 70:
                return 4 + (70 - angle) / 10 * 2  # Escala 4-6
            elif angle > 75:
                return max(0, 3 - (angle - 75) / 10 * 3)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def detect_press_banca(self, landmarks, counter, velocity):
        if landmarks:
            # Puntos clave relevantes
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            
            # Ángulos de codos
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            # Detectar estado y calcular puntuaciones según el estado
            if self.state == "up" and left_elbow_angle <= 70 and right_elbow_angle <= 70:
                # Punto máximo de flexión (contracción)
                self.puntuacion_bajo = self.calculate_contraction_score(left_elbow_angle, right_elbow_angle)
                self.state = "down"
                velocity.start_timing(left_wrist.y)  # Inicia el temporizador

            elif self.state == "down" and left_elbow_angle >= 135 and right_elbow_angle >= 135:
                # Punto máximo de extensión
                self.puntuacion_extension = self.calculate_extension_score(left_elbow_angle, right_elbow_angle)
                self.state = "up"
                self.tiempo_estimado = velocity.stop_timing(left_wrist.y)  # Finalizar tiempo y posición
                # Calcular ROM solo al finalizar una repetición completa
                velocity.update(self.tiempo_estimado)
                
                self.puntuacion_rom = np.mean([self.puntuacion_extension, self.puntuacion_bajo])
                self.ROM.append(self.puntuacion_rom)
            # Actualizar repetición al volver a la extensión
            last_move = counter.state
            self.repeticion = counter.update(left_elbow_angle, right_elbow_angle)
        # Retornar siempre los valores actuales
        return [
            f"Extension: {self.puntuacion_extension:.2f}",
            f"Contraccion: {self.puntuacion_bajo:.2f}",
            f"ROM: {self.puntuacion_rom:.2f}",
            f"Repeticiones: {self.repeticion}",
            f"velocidad: {velocity.get_average_velocity():.2f} m/s"
        ]
            

# Cambia el argumento a la ruta de tu video
video_path = "press_banca3.mp4"  # Reemplaza con la ruta a tu archivo de video
cap = cv2.VideoCapture(video_path)
counter = PressBancaCounter()
velocity = VelocityCalculator()
state = PressBancaState()
RM = Rm(float(input("Introduce el peso con el que realizas el ejercicio: ")))

prev_time = None
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Fin del video o archivo no encontrado.")
        
        break
    
    velocity.frame_high = frame.shape[0]
    # Convierte la imagen a RGB (MediaPipe requiere RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame.flags.writeable = False

    # Procesa la detección de poses
    results = pose.process(rgb_frame)

    # Visualiza los resultados
    rgb_frame.flags.writeable = True
    frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        # Detecta el ejercicio basado en las posiciones
        velocity.fps_original = (cap.get(cv2.CAP_PROP_FPS))
        
        current_time = time.time()
        if prev_time is not None:
            # Calcula el tiempo entre fotogramas
            delta_time = current_time - prev_time
            # Puedes calcular FPS reales del procesamiento
            velocity.actual_fps = 1 / delta_time
        prev_time = current_time
        
        exercise1, exercise2, exercise3, exercise4, exercise5 = state.detect_press_banca(results.pose_landmarks.landmark, counter, velocity)

        frame_count += 1
        # Ajuste dinámico del tamaño del texto según el tamaño del frame
        height, width, _ = frame.shape
        font_scale = width / 950  # Escala el tamaño del texto en función del ancho del video
        font = cv2.FONT_HERSHEY_SIMPLEX

        # Ponemos el texto en la esquina superior izquierda
        cv2.putText(
            frame, exercise1, (10, 40), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise2, (10, 80), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise3, (10, 120), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise4, (10, 160), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise5, (10, 200), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )

    # Muestra la salida
    cv2.imshow('Gym Exercise Analysis', frame)

    # Salir con 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
Epley = RM.Epley(exercise4)
Brzycki  = RM.Brzycki(exercise4)
EpleyReducido = RM.EpleyReducida(exercise4)
rmVelocidad = RM.RmVelocidad(exercise4, velocity.velocities)
reps = RM.RepsToInt(exercise4)
ROM = state.ROM
print(f"repeticiones totales: {reps:.2f}")
print(f"RM con EPLEY: {Epley:.2f} \n", f"RM con BRZYCKI(bueno a repeticiones bajas): {Brzycki:.2f}\n", f"RM con EPLEYREDUCIDO: {EpleyReducido:.2f}\n", f"RM con Velocidad: {rmVelocidad:.2f}\n",  f"Velocidad media: {velocity.get_average_velocity():.2f} m/s")
cap.release()
cv2.destroyAllWindows()

ZeroDivisionError: division by zero

: 

In [63]:
#SENTADILLAS

import cv2
import mediapipe as mp
import numpy as np
import time

# Inicializa MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)


def calculate_angle(a, b, c):
    """Calcula el ángulo entre tres puntos clave."""
    a = np.array([a.x, a.y])  # Primer punto
    b = np.array([b.x, b.y])  # Punto central (vértice)
    c = np.array([c.x, c.y])  # Tercer punto

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

class SentadillasCounter:
    def __init__(self):
        self.reps = 0
        self.state = "up"  # Estado inicial: en extensión completa

    def update(self, left_angle, right_angle):
        # Umbrales para los ángulos
        min_flex_angle = 80  # Punto máximo de flexión
        max_extend_angle = 135  # Punto máximo de extensión

        # Detectar transición de "up" a "down"
        if self.state == "up" and left_angle <= min_flex_angle and right_angle <= min_flex_angle:
            self.state = "down"

        # Detectar transición de "down" a "up"
        elif self.state == "down" and left_angle >= max_extend_angle and right_angle >= max_extend_angle:
            self.reps += 1  # Contar una repetición al volver a "up"
            self.state = "up"

        return self.reps
class Rm:
    def __init__(self, weight):
        self.rm = 0.0
        self.weight = weight   
           
    def Epley(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = (self.weight*newReps*0.033)+self.weight
        return self.rm
    
    def Brzycki(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = self.weight/(1.0278 - (0.0278*newReps))
        return self.rm
    
    def EpleyReducida(self, reps):
        newReps = self.RepsToInt(reps)
        work = 0.03
        suma = 0
        for i in range(5):
            suma += (self.weight*newReps*work)+self.weight
            work -= 0.001
        suma = suma/5
        self.rm = round(suma, 1)
        return self.rm
    
    def RmVelocidad(self, reps, velocidades):
        newReps = self.RepsToInt(reps)
        suma = 0
        
        for velocidad in velocidades:
            suma += self.weight / (1 - (velocidad * 0.03))
        suma = suma/len(velocidades)
        self.rm = round(suma,1)
        return self.rm    
        """#hacer el cambio de la formula
        VR = max(velocidad / 0.278, 0.1)
        suma += (self.weight * (1 + 0.0333 * newReps)) * (1 / VR)
    suma = suma/len(velocidades)
    self.rm = round(suma,1)
    return self.rm"""
    
    def RepsToInt(self, reps):
        return float(reps.split(':')[-1].strip())
    
class VelocityCalculator:
    def __init__(self):
        self.last_time = None
        self.velocities = []
        self.fps_original = 0
        self.actual_fps = 0
        self.frame_high = 0
        # Nuevos atributos para tiempo transcurrido
        self.start_time = None  # Tiempo en posición baja
        self.end_time = None  # Tiempo en posición alta
        self.times = []  # Lista para almacenar los tiempos transcurridos entre eventos
        self.distancia = 0.47

    def update(self, tiempo_estimado):
        """
        Calcula y almacena la velocidad basada en el tiempo entre eventos.
        """
        if tiempo_estimado > 0:  # Prevenir divisiones por cero
            velocidad = 1 / tiempo_estimado  # Velocidad proporcional al tiempo
            velocidad_ajustada = velocidad * (self.fps_original / self.actual_fps)
            self.velocities.append(velocidad)
            return velocidad_ajustada
            #esto es para el caso de utilizar una medida de referencia (longitud del brazo)
            """velocidad = self.distancia / tiempo_estimado 
            self.velocities.append(velocidad)
            return velocidad"""

        #return None

    def start_timing(self, wrist_position_y):
        """Inicia el temporizador y registra la posición de la muñeca en el punto bajo."""
        self.start_time = time.time()
        self.start_position_y = wrist_position_y * self.frame_high
        
    def stop_timing(self, wrist_position_y):
        """Detiene el temporizador y registra la posición de la muñeca en el punto alto."""
        self.end_time = time.time()
        if self.start_time is not None and self.end_time is not None:
            elapsed_time = self.end_time - self.start_time

            
            corrected_elapsed_time = (elapsed_time * self.actual_fps) / self.fps_original
            # Almacena el tiempo corregido para análisis posterior
            self.times.append(corrected_elapsed_time)

            self.start_time = None  # Reinicia para la próxima repetición
            return corrected_elapsed_time

        return None, None, None

    def get_average_time(self):
        """Calcula el tiempo promedio entre eventos registrados."""
        if self.times:
            return np.mean(self.times)
        return 0.0

    def get_average_velocity(self):
        """Calcula la velocidad media de todas las repeticiones registradas."""
        if self.velocities:
            return np.mean(self.velocities)
        return 0.0

    def set_actual_fps(self, actual_fps):
        """Establece el FPS actual, útil para ajustar las velocidades cuando se ralentiza el video."""
        self.actual_fps = actual_fps

class SentadillasState: 
    def __init__(self):
        self.puntuacion_extension = 0.0
        self.puntuacion_bajo = 0.0
        self.puntuacion_rom = 0.0
        self.repeticion = 0
        self.state = "up"  # Estado inicial (extensión máxima)
        self.tiempo_estimado = 0
        self.ROM = []

    def calculate_extension_score(self, left_angle, right_angle):
        """Calcula la puntuación de la extensión en el punto máximo con ajustes más relajados."""
        def score_angle(angle):
            if 150 <= angle <= 180:
                return 9 + (angle - 150) / 20  # Escala 9-10
            elif 130 <= angle < 150:
                return 7 + (angle - 130) / 20 * 2  # Escala 7-8
            elif 110 <= angle < 130:
                return 4 + (angle - 110) / 20 * 3  # Escala 4-6
            elif angle < 110:
                return max(0, (angle - 90) / 20 * 4)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def calculate_contraction_score(self, left_angle, right_angle):
        """Calcula la puntuación de la contracción en el punto máximo con ajustes."""
        def score_angle(angle):
            if 1 <= angle <= 75:
                score = 9 + (75 - angle) / 15
                return min(score, 10)
            elif 75 < angle <= 85:
                return 7 + (85 - angle) / 10 * 2  # Escala 7-8
            elif 85 < angle <= 95:
                return 4 + (95 - angle) / 10 * 3  # Escala 4-6
            elif angle > 95:
                return max(0, 3 - (95 - angle) / 10 * 2)  # Escala 0-3
            return 0
        
        score_izquierda = score_angle(left_angle)
        score_derecha = score_angle(right_angle)

        if abs(score_izquierda - score_derecha) >= 10:
            if score_izquierda < score_derecha:
                return (score_izquierda * 0.75 + score_derecha * 0.25)
            else:
                return (score_izquierda* 0.25 + score_derecha* 0.75)
        
        return (score_angle(left_angle) + score_angle(right_angle)) / 2
        

    def detect_squat(self, landmarks, counter, velocity):
        if landmarks:
            # Puntos clave relevantes
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

            # Ángulos de caderas y rodillas
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            
            # Detectar estado y calcular puntuaciones según el estado
            if self.state == "up" and left_hip_angle <= 95 and right_hip_angle <= 95:
                # Punto máximo de flexión (abajo)
                self.puntuacion_bajo = self.calculate_contraction_score(left_hip_angle, right_hip_angle)
                self.state = "down"
                velocity.start_timing(left_ankle.y)  # Inicia el temporizador

            elif self.state == "down" and left_hip_angle >= 135 and right_hip_angle >= 135:
                # Punto máximo de extensión (arriba)
                self.puntuacion_extension = self.calculate_extension_score(left_hip_angle, right_hip_angle)
                self.state = "up"
                self.tiempo_estimado = velocity.stop_timing(left_ankle.y)  # Finalizar tiempo y posición
                velocity.update(self.tiempo_estimado)

                self.puntuacion_rom = np.mean([self.puntuacion_extension, self.puntuacion_bajo])
                self.ROM.append(self.puntuacion_rom)

            # Actualizar repetición al volver a la extensión
            last_move = counter.state
            self.repeticion = counter.update(left_hip_angle, right_hip_angle)

        # Retornar siempre los valores actuales
        return [
            f"Extension: {self.puntuacion_extension:.2f}",
            f"Contraccion: {self.puntuacion_bajo:.2f}",
            f"ROM: {self.puntuacion_rom:.2f}",
            f"Repeticiones: {self.repeticion}",
            f"velocidad: {velocity.get_average_velocity():.2f} m/s"
        ]

            

# Cambia el argumento a la ruta de tu video
video_path = "sentadillas1.mp4"  # Reemplaza con la ruta a tu archivo de video
cap = cv2.VideoCapture(video_path)
counter = SentadillasCounter()
velocity = VelocityCalculator()
state = SentadillasState()
RM = Rm(float(input("Introduce el peso con el que realizas el ejercicio: ")))

prev_time = None
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Fin del video o archivo no encontrado.")
        
        break
    
    velocity.frame_high = frame.shape[0]
    # Convierte la imagen a RGB (MediaPipe requiere RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame.flags.writeable = False

    # Procesa la detección de poses
    results = pose.process(rgb_frame)

    # Visualiza los resultados
    rgb_frame.flags.writeable = True
    frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        # Detecta el ejercicio basado en las posiciones
        velocity.fps_original = (cap.get(cv2.CAP_PROP_FPS))
        
        current_time = time.time()
        if prev_time is not None:
            # Calcula el tiempo entre fotogramas
            delta_time = current_time - prev_time
            # Puedes calcular FPS reales del procesamiento
            velocity.actual_fps = 1 / delta_time
        prev_time = current_time
        
        exercise1, exercise2, exercise3, exercise4, exercise5 = state.detect_squat(results.pose_landmarks.landmark, counter, velocity)

        frame_count += 1
        # Ajuste dinámico del tamaño del texto según el tamaño del frame
        height, width, _ = frame.shape
        font_scale = width / 950  # Escala el tamaño del texto en función del ancho del video
        font = cv2.FONT_HERSHEY_SIMPLEX

        # Ponemos el texto en la esquina superior izquierda
        cv2.putText(
            frame, exercise1, (10, 40), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise2, (10, 80), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise3, (10, 120), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise4, (10, 160), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise5, (10, 200), font, font_scale, (0, 0, 255), 2, cv2.LINE_AA
        )

    # Muestra la salida
    cv2.imshow('Gym Exercise Analysis', frame)

    # Salir con 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
Epley = RM.Epley(exercise4)
Brzycki  = RM.Brzycki(exercise4)
EpleyReducido = RM.EpleyReducida(exercise4)
rmVelocidad = RM.RmVelocidad(exercise4, velocity.velocities)
reps = RM.RepsToInt(exercise4)
ROM = state.ROM
print(f"repeticiones totales: {reps:.2f}")
print(f"RM con EPLEY: {Epley:.2f} \n", f"RM con BRZYCKI(bueno a repeticiones bajas): {Brzycki:.2f}\n", f"RM con EPLEYREDUCIDO: {EpleyReducido:.2f}\n", f"RM con Velocidad: {rmVelocidad:.2f}\n",  f"Velocidad media: {velocity.get_average_velocity():.2f} m/s")
cap.release()
cv2.destroyAllWindows()

Angulo izquierdo: 67.23268985528378
Angulo derecho: 76.17763878210562
Angulo izquierdo: 67.67398860050336
Angulo derecho: 81.39330490302653
Angulo izquierdo: 62.04626359405864
Angulo derecho: 2.588829355874361
Fin del video o archivo no encontrado.
repeticiones totales: 3.00
RM con EPLEY: 109.90 
 RM con BRZYCKI(bueno a repeticiones bajas): 105.89
 RM con EPLEYREDUCIDO: 108.40
 RM con Velocidad: 101.10
 Velocidad media: 0.37 m/s
